In [2]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns
import csv
from scipy.io import arff
from sklearn import datasets
from sklearn.cluster import KMeans
import numpy as np
from kmodes.kmodes import KModes
from kmodes.util.dissim import ng_dissim


In [92]:
#data = arff.loadarff('universityPolicyTrainingData.arff')
permit = pd.read_csv('permittedtest.csv')
denied = pd.read_csv('deniedtest.csv')
#df = pd.DataFrame(permit)
refine = pd.read_csv('step2ExtractedRulesfalseNegatives.csv')

df = pd.DataFrame(refine)
#print(denied)
act = pd.concat([permit,denied])
#print(data)

df = pd.DataFrame(refine)
#df= df.drop(columns ='Unnamed: 9')

print()
print(df.describe())
df.head()


       ischair    crs odepartment       action position       type  \
count      982    982         982          982      982        982   
unique       3      5           5            2        4          4   
top       true  ee601          cs  changescore  faculty  gradebook   
freq       850    273         611          500      792        639   

       udepartment crstaught crstaken  
count          982       982      982  
unique           5         5        5  
top      registrar     ee101    cs602  
freq           270       290      213  


,ischair,crs,odepartment,action,position,type,udepartment,crstaught,crstaken
0,true,ee601,cs,read,advisor,transcript,registrar,ee101,cs601
1,true,cs602,registrar,read,faculty,application,registrar,ee101,cs602
2,na,ee601,cs,read,faculty,gradebook,ee,ee601,cs602
3,true,ee101,registrar,read,student,gradebook,registrar,cs602,ee101
4,true,cs602,cs,changescore,faculty,gradebook,cs,cs101,cs101


In [93]:
#print(df.describe())

df.isnull().values.any()
df["position"] = df["position"].astype('category')
df["udepartment"] = df["udepartment"].astype('category')
df["crstaken"] = df["crstaken"].astype('category')
df["crstaught"] = df["crstaught"].astype('category')
df["ischair"] = df["ischair"].astype('category')
df["type"] = df["type"].astype('category')
df["crs"] = df["crs"].astype('category')
df["odepartment"] = df["odepartment"].astype('category')
df["action"] = df["action"].astype('category')

act.isnull().values.any()
act["position"] = act["position"].astype('category')
act["udepartment"] = act["udepartment"].astype('category')
act["crstaken"] = act["crstaken"].astype('category')
act["crstaught"] = act["crstaught"].astype('category')
act["ischair"] = act["ischair"].astype('category')
act["type"] = act["type"].astype('category')
act["crs"] = act["crs"].astype('category')
act["odepartment"] = act["odepartment"].astype('category')
act["action"] = act["action"].astype('category')


df.head(30)
print(df.dtypes)

ischair        category
crs            category
odepartment    category
action         category
position       category
type           category
udepartment    category
crstaught      category
crstaken       category
dtype: object


In [94]:
# Attributes Frequencies
datafreq = pd.DataFrame()
for col in df.columns:
    datafreq=df[col].value_counts()
    #print (datafreq)
    
#print(datafreq)



In [95]:
# Correlation Function

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Writing Data Correlation in a CSV
with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
    
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()



/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [96]:
data = df
k=5
datasize= len(act.index)
print(datasize)
km = KModes(n_clusters=k, init='Cao', n_init=1, verbose=2)
df_dummy = pd.get_dummies(df)

clusters = km.fit_predict(data)
df_dummy['clusters'] = clusters

# Print the cluster centroids

centroids=km.cluster_centroids_

print(centroids)

12000
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 64, cost: 3269.0
Run 1, iteration: 2/100, moves: 113, cost: 3212.0
Run 1, iteration: 3/100, moves: 5, cost: 3212.0
[['true' 'ee601' 'cs' 'changescore' 'faculty' 'gradebook' 'registrar'
  'ee101' 'cs602']
 ['true' 'ee101' 'ee' 'read' 'faculty' 'transcript' 'ee' 'ee101' 'ee101']
 ['true' 'cs101' 'cs' 'read' 'faculty' 'gradebook' 'cs' 'cs602' 'cs101']
 ['true' 'ee601' 'registrar' 'read' 'faculty' 'gradebook' 'registrar'
  'ee601' 'ee601']
 ['na' 'ee101' 'ce' 'read' 'faculty' 'gradebook' 'ce' 'ee101' 'cs602']]


In [97]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = data.index.values
cluster_map['cluster'] = km.labels_

#valuescount=pd.DataFrame()
#temp=[]

for i in range(0, 9):
    ind=cluster_map.cluster == i
    #print(ind)

In [98]:
cluster_map
ndf=pd.concat([data,cluster_map], axis=1)
ndf=ndf.drop(columns='data_index')
ndf

,ischair,crs,odepartment,action,position,type,udepartment,crstaught,crstaken,cluster
0,true,ee601,cs,read,advisor,transcript,registrar,ee101,cs601,0
1,true,cs602,registrar,read,faculty,application,registrar,ee101,cs602,0
2,na,ee601,cs,read,faculty,gradebook,ee,ee601,cs602,0
3,true,ee101,registrar,read,student,gradebook,registrar,cs602,ee101,3
4,true,cs602,cs,changescore,faculty,gradebook,cs,cs101,cs101,2
5,true,cs601,ee,read,advisor,application,ee,cs101,cs601,1
6,false,ee601,admissions,read,faculty,gradebook,ee,ee601,ee601,3
7,na,ee101,cs,read,faculty,application,admissions,ee101,cs101,4
8,na,ee101,registrar,read,faculty,gradebook,admissions,ee101,cs602,4
9,true,ee101,admissions,read,student,transcript,registrar,cs602,ee101,1


In [99]:
#centroids columns=['ischair','crs','odepartment','action','position',
                             # 'type','udepartment','crstaught','crstaken']
ndf.groupby('cluster').count()

,ischair,crs,odepartment,action,position,type,udepartment,crstaught,crstaken
cluster,,,,,,,,,
0,465,465,465,465,465,465,465,465,465
1,200,200,200,200,200,200,200,200,200
2,124,124,124,124,124,124,124,124,124
3,141,141,141,141,141,141,141,141,141
4,52,52,52,52,52,52,52,52,52


In [109]:
# Calculating Attributes Effectevness in each cluster
temp = pd.DataFrame()
datafreq = pd.DataFrame()
l=[]
extract = pd.DataFrame()
rules = pd.DataFrame(columns=['ischair','crs','odepartment','action','position',
                              'type','udepartment','crstaught','crstaken'])
print(datafreq)
clusTemp = pd.DataFrame()
for i in range(k):
    print()
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    clussize= len(clusTemp.index)
    #print(clussize)
    for col in df.columns:
        datafreq = act[col].value_counts().to_dict()
        #datafreq= pd.DataFrame.from_dict(datafreq, )
        temp=clusTemp[col].value_counts().to_dict()
        #temp= pd.DataFrame.from_dict(temp)
        #print(temp)
        #print(datafreq)
        for key, value in datafreq.items():
            #if key not in temp.keys():
                #print (key, 'is not in the cluster')
        #    datafreq["count"] = df.loc[col == attr].size()
            #print(key)
            x = value/datasize
            #print(x)
            for key1, value1 in temp.items():
                
                if key == key1:
                    y = value1/clussize
                    #print(y)
                    
                    if y-x >= 0.40:
                        #print('the effectevniess of ', key, y, x, 'is: ', y-x)
                        #l.append(key)
                        #print(col)
                        print(col,'=',key,',',end="")
                    if y-x < -0.3:
                         print (col,'!=',key,',',end="")
        #rules.append(pd.DataFrame([col,l]))
        #rules.loc[i, [col]] = pd.DataFrame(l)
        #rules=pd.DataFrame(columns[col])
        #rules[col]=pd.DataFrame(l)
        #print(rules)
            

#for i in range(k): 
    


Empty DataFrame
Columns: []
Index: []

CLUSTER NUMBER 0
ischair = true ,ischair != false ,odepartment = cs ,action = changescore ,position = faculty ,type = gradebook ,
CLUSTER NUMBER 1
ischair = true ,action = read ,
CLUSTER NUMBER 2
ischair = true ,odepartment = cs ,udepartment = cs ,
CLUSTER NUMBER 3
crs = ee601 ,action = read ,crstaught = ee601 ,
CLUSTER NUMBER 4
ischair != true ,ischair = na ,crs = ee101 ,action = read ,position = faculty ,crstaught = ee101 ,

In [108]:
datacorr = pd.DataFrame()
for i in range(k):
    print("=============================================================")
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    for col in df.columns:
        if col in l:
    #writer = csv.writer(f, dialect='excel')
            for col2 in df.columns:
                if col2 in l:
                    confusion_matrix = pd.crosstab(act[col], act[col2]).as_matrix()
                    datacorr = cramers_v(confusion_matrix)
                    #print('correlation between ',col,'and', col2,'is : ',datacorr)
                    temp=clusTemp[col].value_counts().to_dict()    
                    confusion_matrix = pd.crosstab(clusTemp[col], clusTemp[col2]).as_matrix()
                    temp = cramers_v(confusion_matrix)
                    #print (temp, datacorr)
                    if temp > datacorr :
                        #print (temp, datacorr)
                        c= temp - datacorr
                        if c >= 0.40:
                            print (temp, datacorr)
                            print(col,'==', col2,'is : ', c )
                    else:
                        c= datacorr - temp 
                        if c >= 0.10:
                    
                            print (temp, datacorr)
                            print(col,'!==', col2,'is : ', c )

CLUSTER NUMBER 0
CLUSTER NUMBER 1
CLUSTER NUMBER 2
CLUSTER NUMBER 3
CLUSTER NUMBER 4


In [337]:
from sklearn.decomposition import PCA
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
        distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
        return distances
#clusters=km.fit_predict(data2D)
#centroids = km.cluster_centers_
#data2D=PCA(n_components=2).fit_transform(data)
distances = []
for i, (cx, cy) in enumerate(centroids):
    mean_distance = k_mean_distance(data2D, cx, cy, i, clusters)
    distances.append(mean_distance)

print(distances)

ValueError: could not convert string to float: 'true'

In [211]:
# Correlation in Each Cluster

with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
        
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()



/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
